<a href="https://colab.research.google.com/github/Aldarion96/Projects/blob/main/Alza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

df_retail = pd.read_excel (r'/content/sample_data/Online Retail.xlsx')
print (df_retail)

       InvoiceNo StockCode  ... CustomerID         Country
0         536365    85123A  ...    17850.0  United Kingdom
1         536365     71053  ...    17850.0  United Kingdom
2         536365    84406B  ...    17850.0  United Kingdom
3         536365    84029G  ...    17850.0  United Kingdom
4         536365    84029E  ...    17850.0  United Kingdom
...          ...       ...  ...        ...             ...
541904    581587     22613  ...    12680.0          France
541905    581587     22899  ...    12680.0          France
541906    581587     23254  ...    12680.0          France
541907    581587     23255  ...    12680.0          France
541908    581587     22138  ...    12680.0          France

[541909 rows x 8 columns]


**Úprava datasetu:**

*čištění*
-řádky s Description pouze znaky, wrong, wrongly, 
- řádky kde není C ale je quantity záporný
- prázdné pole Description s nulovou cenou
- A = Adjust bad debt
- C = zrušené nákupy (10 000 řádků)

*Vytvořit*
- sloupec flag
-sloupec profit (quantity * price)



*Vizualizace*
- ze které země se nakupuje nejvíce
- jaký produkt se nakupuje nejvíce
celkový profit/ztráta / za kvartál
 

**Čištění**

**Odstranení v sloupci *Description***

In [6]:
clear_df = df_retail[~df_retail.Description.str.contains("\?|wrong|damage|crushed|missing|destroyed|lost|mouldy|wet|thrown|smashed|check|Dotcom|Printing|broken|counted|problem|away|ebay|incorrect", na=False,case=False)]

In [7]:
#clear_df.query("Description == 'wrong code'")

**1. Jaký produkt se prodává nejvíce:**

počet nákupů

In [8]:
sells_df = clear_df.assign(flag = 1)
sells_df[['Description','StockCode','flag']].groupby(['StockCode','Description']).count().sort_values('flag',ascending = False)

,,flag
StockCode,Description,
85123A,WHITE HANGING HEART T-LIGHT HOLDER,2302
22423,REGENCY CAKESTAND 3 TIER,2200
85099B,JUMBO BAG RED RETROSPOT,2159
47566,PARTY BUNTING,1727
20725,LUNCH BAG RED RETROSPOT,1638
...,...,...
84968f,SET OF 16 VINTAGE SKY BLUE CUTLERY,1
35650,VINTAGE BEAD PINK JEWEL STAND,1
35832,WOOLLY HAT SOCK GLOVE ADVENT STRING,1


**počet prodaných kusů**

In [9]:
sells_df[['Description','StockCode','Quantity']].groupby(['StockCode','Description']).sum().sort_values('Quantity',ascending = False).head()

,,Quantity
StockCode,Description,
84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
85099B,JUMBO BAG RED RETROSPOT,47363
84879,ASSORTED COLOUR BIRD ORNAMENT,36381
22197,POPCORN HOLDER,36334
21212,PACK OF 72 RETROSPOT CAKE CASES,36039


**Ve které zemi se prodává nejvíce:**

**Vytvoření pmocných sloupců.**


- **total** = celkový výnos/ztráta
- **flag** = počet zakoupení
- **quantity** = čeho se prodalo nejvíce / nejméně




In [ ]:
df_countries = df_retail[['Country']].assign(flag = 1).groupby('Country').count().sort_values('flag',ascending = False)
df_countries

**Zisk / Ztráta**

Tabulka s vypočítanou ziskovostí či ztrátovostí

In [33]:
df_profit = df_retail.assign(total = (df_retail.Quantity * df_retail.UnitPrice))
df_profit.head(3)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00


In [34]:
df_profit = df_profit[['Description','StockCode','total']].groupby(['StockCode','Description']).sum().sort_values('total',ascending = False)

In [37]:
df_profit = df_profit.query("total != 0").sort_values('total', ascending = False)
df_profit

,,total
StockCode,Description,
DOT,DOTCOM POSTAGE,206245.480
22423,REGENCY CAKESTAND 3 TIER,164762.190
47566,PARTY BUNTING,98302.980
85123A,WHITE HANGING HEART T-LIGHT HOLDER,97715.990
85099B,JUMBO BAG RED RETROSPOT,92356.030
...,...,...
BANK CHARGES,Bank Charges,-7175.639
CRUK,CRUK Commission,-7933.430
B,Adjust bad debt,-11062.060


**Ziskovost celkově za dny/měsíce/kvartály**

In [16]:
df_days = df_retail.assign(total = (df_retail.Quantity * df_retail.UnitPrice))

In [18]:
df_days.index = pd.to_datetime(df_days['InvoiceDate'],format='%m/%d/%y %I:%M%p')

In [19]:
df_days[['total']].groupby(pd.Grouper(freq='d')).sum().sort_values('InvoiceDate')


,total
InvoiceDate,
2010-12-01,58635.56
2010-12-02,46207.28
2010-12-03,45620.46
2010-12-04,0.00
2010-12-05,31383.95
...,...
2011-12-05,57751.32
2011-12-06,54228.37
2011-12-07,75076.22


In [21]:
df_months = df_days.query("InvoiceDate >= '2011-01-01'")
df_months[['total']].groupby(pd.Grouper(freq='m')).sum().sort_values('InvoiceDate')


,total
InvoiceDate,
2011-01-31,560000.260
2011-02-28,498062.650
2011-03-31,683267.080
2011-04-30,493207.121
2011-05-31,723333.510
2011-06-30,691123.120
2011-07-31,681300.111
2011-08-31,682680.510
2011-09-30,1019687.622


osekat na jeden rok

In [23]:
df_quartal = df_days.query("InvoiceDate >= '2011-01-01'")

In [24]:
df_quartal[['total']].groupby(pd.Grouper(freq='q')).sum().sort_values('InvoiceDate')

,total
InvoiceDate,
2011-03-31,1741329.990
2011-06-30,1907663.751
2011-09-30,2383668.243
2011-12-31,2966128.930
